In [792]:
import pandas as pd
import dtale
from pandas import DataFrame

In [793]:
def dataframe_difference(df1: DataFrame, df2: DataFrame, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    diff_df.to_csv('data/diff.csv')
    return diff_df

In [794]:
#Lets add country code
alpha3 = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1')[1]
alpha3 = alpha3.iloc[:,[0,2]]
alpha3 = alpha3.rename(columns={alpha3.columns[0]: 'Country', alpha3.columns[1]:'A3'})
alpha3.head(n=3)

,Country,A3
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB


## Notes:
For merging we use the outer option, to not loos too many entries, after splitting off the test set, we apply the MICE algorithm to the training set to replace nan values. This algorithm does preserve the overall distribution of the data:
https://towardsdatascience.com/whats-the-best-way-to-handle-nan-values-62d50f738fc

# Number of refugees 

In [795]:
num_ref0 = pd.read_csv('Data/asylum_seekers/population.csv', skiprows=14)
num_ref0 = num_ref0[num_ref0.columns[[0,1,2,5]]]
num_ref0 = num_ref0.rename(
    columns={num_ref0.columns[1]: 'Country',num_ref0.columns[2]: 'A3', num_ref0.columns[3]:'Refugees'})
num_ref0.head(n=1)

,Year,Country,A3,Refugees
0,1996,Afghanistan,AFG,2674234


## Drop some rows and columns

In [796]:
null_mask = num_ref0['A3'].isnull()
num_ref0[null_mask]#These are high numbers for unknown countries, but we have to drop them
num_ref0 = num_ref0[~null_mask]
num_ref = num_ref0[num_ref0.columns[[0,2,3]]]#
print('unique values:', num_ref['A3'].nunique())
num_ref.head()

unique values: 210


,Year,A3,Refugees
0,1996,AFG,2674234
1,1996,ALB,5784
2,1996,DZA,2245
3,1996,AGO,249686
4,1996,EGY,1198


In [797]:
#are there any nan values?
num_ref[num_ref['Refugees'].isnull()]# no nan values

,Year,A3,Refugees


# Total populations numbers 

In [798]:
pop0 = pd.read_csv('Data/Total_Population/Population.csv', skiprows=4, encoding = "ISO-8859-1")
#exclude years until 1996
rem_cols = [c for c in pop0.columns if c[:2] == "19"][:-4]
pop0.drop(columns = rem_cols, inplace = True)
#Drop indicator code and indicator name
rem_cols = [c for c in pop0.columns if c[:3] == "Ind"]
pop0.drop(columns = rem_cols, inplace = True)
#last two columns are NaN
pop0.drop(columns = pop0.columns[-2:], inplace = True)
#now convert columns into rows
pop0 = pop0.melt(id_vars=["Country Name", "Country Code"], 
        var_name="Year", 
        value_name="Population")
pop0 = pop0.sort_values(by=['Country Name', 'Year'])# we can sort by country or by year
pop0 = pop0.rename(
    columns={pop0.columns[0]: 'Country',pop0.columns[1]: 'A3'})
pop0.head(n = 2)

,Country,A3,Year,Population
1,Afghanistan,AFG,1996,18853437.0
265,Afghanistan,AFG,1997,19357126.0


In [799]:
#turn years into integers
pop0['Year'] = pop0['Year'].apply(pd.to_numeric, errors = 'coerce')
#are there country codes which are nan?
np.count_nonzero(pop0['A3'].isnull())
#nope we can drop the Country column
pop = pop0.drop(columns='Country')
print('unique values: ', pop['A3'].nunique())
pop.head(n=2)

unique values:  264


,A3,Year,Population
1,AFG,1996,18853437.0
265,AFG,1997,19357126.0


## Merge

In [800]:
Input0 = pop.merge(num_ref, how = 'outer', on = ['A3', 'Year']) #try outer join
#exclude countries the have nans for the whole period
Input0 = Input0[~(Input0.groupby('A3')['Refugees'].transform('count')==0)]
Input0.head(n=4)

,A3,Year,Population,Refugees
0,AFG,1996,18853437.0,2674234.0
1,AFG,1997,19357126.0,2676675.0
2,AFG,1998,19737765.0,2667118.0
3,AFG,1999,20170844.0,2601690.0


In [801]:
a3_pop = pop.A3.unique()
a3_ref = num_ref.A3.unique()
set_difference = set(a3_pop) - set(a3_ref)
list_difference = list(set_difference)
alpha3[alpha3.A3.isin(list_difference)]

,Country,A3
4,American Samoa,ASM
12,Aruba,ABW
73,Faroe Islands,FRO
78,French Polynesia,PYF
87,Greenland,GRL
90,Guam,GUM
108,Isle of Man,IMN
165,Northern Mariana Islands,MNP
189,Saint Martin (French part),MAF
201,Sint Maarten (Dutch part),SXM


# Disasters 

In [802]:
#Here we will use only the number of deaths summed up over one year, Nan values are replaced with zero
disasters0 = pd.read_excel('Data/Disaster/disasters.xlsx', engine='openpyxl', skiprows = 6)
disasters0['Country Code'] = [code[-3:] for code in disasters0['Dis No']]
#print(disasters[disasters['Country Code']=='SYR'])
#move country code to the first position
cols = disasters0.columns.tolist()
cols.insert(1, cols.pop(-1))# move it to the right position
disasters0 = disasters0[cols]
disasters0.head(2)

,Dis No,Country Code,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,1996-0003-BRA,BRA,1996,3,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,...,4.0,15.0,NaN,NaN,7000.0,7000.0,NaN,NaN,NaN,61.35163
1,1996-0116-CAN,CAN,1996,116,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,...,22.0,10.0,NaN,NaN,12000.0,12000.0,NaN,254900.0,729600.0,61.35163


Lets separate Natural and non-natural disasters

In [803]:
natural_mask = disasters0['Disaster Group']=='Natural'
disasters_nat0 = disasters0[natural_mask]
disasters_tec0 = disasters0[~natural_mask]
disasters_tec0.head(2)

,Dis No,Country Code,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
6,1996-0039-AFG,AFG,1996,39,Technological,Technological,Industrial accident,Explosion,NaN,Ammunition store,...,16.0,60.0,125.0,NaN,NaN,125.0,NaN,NaN,NaN,61.35163
12,1996-0146-AUS,AUS,1996,146,Technological,Technological,Transport accident,Air,NaN,Military Helicopter,...,13.0,18.0,10.0,50.0,NaN,60.0,NaN,NaN,NaN,61.35163


## Natural Disasters

In [804]:
#We sum all the disasters up for one year
#to get the best representation we will use total deaths and total affected since deaths are not included in that
tot_aff_nat = disasters_nat0.groupby(['Country Code','Year'])['Total Affected'].sum().reset_index()
tot_deaths_nat = disasters_nat0.groupby(['Country Code','Year'])['Total Deaths'].sum().reset_index()
disasters_nat = pd.concat([tot_aff_nat, tot_deaths_nat], axis = 1)
#remove columns that occer twice
disasters_nat = tot_aff_nat.join(tot_deaths_nat['Total Deaths'])

disasters_nat = disasters_nat.rename(
    columns={disasters_nat.columns[0]: 'A3', disasters_nat.columns[2]: 'Tot Affected Nat',
            disasters_nat.columns[3]: 'Tot Deaths Nat'})

disasters_nat.head(n = 2)

,A3,Year,Tot Affected Nat,Tot Deaths Nat
0,AFG,1996,13105.0,70.0
1,AFG,1997,20750.0,175.0


## Non-natural disasters

In [805]:
#We sum all the disasters up for one year
#to get the best representation we will use total deaths and total affected since deaths are not included in that
tot_aff_tec = disasters_tec0.groupby(['Country Code','Year'])['Total Affected'].sum().reset_index()
tot_deaths_tec = disasters_tec0.groupby(['Country Code','Year'])['Total Deaths'].sum().reset_index()
disasters_tec = pd.concat([tot_aff_tec, tot_deaths_tec], axis = 1)
#remove columns that occer twice
disasters_tec = tot_aff_tec.join(tot_deaths_tec['Total Deaths'])

disasters_tec = disasters_tec.rename(
    columns={disasters_tec.columns[0]: 'A3', disasters_tec.columns[2]: 'Tot Affected Tec',
            disasters_tec.columns[3]: 'Tot Deaths Tec' })

disasters_tec.head(n = 2)

,A3,Year,Tot Affected Tec,Tot Deaths Tec
0,AFG,1996,125.0,60.0
1,AFG,1997,80.0,54.0


## Merge Nat/non-Nat

In [806]:
disasters_all = disasters_nat.merge(disasters_tec, how = 'outer', on = ['A3', 'Year'])
disasters['Year'] = disasters['Year'].apply(pd.to_numeric, errors = 'coerce')#years should be integers
disasters_all.head(3)

,A3,Year,Tot Affected Nat,Tot Deaths Nat,Tot Affected Tec,Tot Deaths Tec
0,AFG,1996,13105.0,70.0,125.0,60.0
1,AFG,1997,20750.0,175.0,80.0,54.0
2,AFG,1998,165836.0,7308.0,0.0,45.0


## Merge with input 

### Since so many values are missing, its better to exclude disasters!

In [807]:
Input1 = Input0#disasters_all.merge(Input0, how = 'outer', on = ['A3', 'Year'])
#print('unique val: ',Input1['A3'].nunique())
Input1.head(n=2)

,A3,Year,Population,Refugees
0,AFG,1996,18853437.0,2674234.0
1,AFG,1997,19357126.0,2676675.0


# Human Development Index 

In [808]:
hdi0 = pd.read_csv('Data/HDI/HDI.csv', skiprows=5,encoding = "ISO-8859-1")
#Remove all unnamed columns
cols = [c for c in hdi0.columns if (c[:7] != 'Unnamed' and c!='HDI Rank')]
hdi0 = hdi0[cols]
hdi0.head(n=2)

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.302,0.307,0.316,0.312,0.307,0.331,0.335,0.339,0.344,...,0.472,0.477,0.489,0.496,0.500,0.500,0.502,0.506,0.509,0.511
1,Albania,0.650,0.631,0.615,0.618,0.624,0.637,0.646,0.645,0.655,...,0.745,0.764,0.775,0.782,0.787,0.788,0.788,0.790,0.792,0.795


## Reshape 

In [809]:
year_cols = hdi0.columns[1:]
hdi0[year_cols] = hdi0[year_cols].apply(pd.to_numeric, errors = 'coerce')
#hdi1 = hdi0.T.fillna(hdi0.mean(axis=1)).T #Mean should be computed after separating test val and train
#now convert columns into rows
hdi1 = hdi0.melt(id_vars=["Country"], 
        var_name="Year", 
        value_name="HDI")
hdi1 = hdi1.sort_values(by=['Country', 'Year'])
hdi1.head(n=2)

,Country,Year,HDI
0,Afghanistan,1990,0.302
207,Afghanistan,1991,0.307


## Remove leading and trailing spaces

In [810]:
hdi1['Country'].unique()[:5] #some countries have white spaces in the name!!!
hdi1 = hdi1.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#hdi1['Country'] = hdi1['Country'].str.replace(' ', '',1)#replace that

In [811]:
#merge to see which country entries do not match
mergedStuff = pd.merge(hdi1, alpha3, on=['Country'], how='inner')
mergedStuff.head()
merged_countries = mergedStuff['Country'].unique()

In [812]:
#find other countries:
countries_in_both_mask= hdi1['Country'].isin(merged_countries)
diff_countries = hdi1[~countries_in_both_mask]['Country'].unique()
print(diff_countries)
#the last 17 entries should be removed since these are not countries
diff_countries = diff_countries[:-18]
print(diff_countries)

['Congo (Democratic Republic of the)' 'Eswatini (Kingdom of)'
 'Hong Kong, China (SAR)' 'Korea (Republic of)' 'Moldova (Republic of)'
 'Tanzania (United Republic of)' 'United Kingdom' 'United States'
 'Arab States' 'Developing Countries' 'East Asia and the Pacific'
 'Europe and Central Asia' 'High human development' 'Human Development'
 'Latin America and the Caribbean' 'Least Developed Countries'
 'Low human development' 'Medium human development'
 'Organization for Economic Co-operation and Development' 'Regions'
 'Small Island Developing States' 'South Asia' 'Sub-Saharan Africa'
 'Very high human development' 'World' nan]
['Congo (Democratic Republic of the)' 'Eswatini (Kingdom of)'
 'Hong Kong, China (SAR)' 'Korea (Republic of)' 'Moldova (Republic of)'
 'Tanzania (United Republic of)' 'United Kingdom' 'United States']


## Add A3 

In [813]:
#Countries that have a different string in hdi and A3 table
A3_code = [
    'Congo, Democratic Republic of the', 'Eswatini', 'Hong Kong', 
    'Korea, Republic of', 'Moldova, Republic of',
    'Tanzania, United Republic of', 
    'United Kingdom of Great Britain and Northern Ireland','United States of America']

In [814]:
#Replace non-matching countries with entries from the alpha3 df
di = dict(zip(diff_countries,A3_code))
hdi1['Country'] = hdi1['Country'].map(di).fillna(hdi1['Country'])
hdi1.head(n=2)

,Country,Year,HDI
0,Afghanistan,1990,0.302
207,Afghanistan,1991,0.307


In [815]:
#Lets add ISO code
#Lets add country code
hdi = hdi1.merge(alpha3, on = 'Country')#Merge by country such that now ISO is in the table
hdi = hdi.iloc[:,1:]#Deleta Country column
hdi.head(n=2)

,Year,HDI,A3
0,1990,0.302,AFG
1,1991,0.307,AFG


In [816]:
#turn years into integers
hdi['Year'] = hdi['Year'].apply(pd.to_numeric, errors = 'coerce')
hdi_year_mask = hdi['Year']>=1996
hdi = hdi[hdi_year_mask]
Input2 = hdi.merge(Input1, how = 'outer', on = ['A3', 'Year'])
print('unique val: ',Input2['A3'].nunique())
Input2.head()

unique val:  210


,Year,HDI,A3,Population,Refugees
0,1996,0.335,AFG,18853437.0,2674234.0
1,1997,0.339,AFG,19357126.0,2676675.0
2,1998,0.344,AFG,19737765.0,2667118.0
3,1999,0.348,AFG,20170844.0,2601690.0
4,2000,0.350,AFG,20779953.0,3587327.0


# Temperature 

In [817]:
temp0 = pd.read_csv('Data/Temperature/Temp_noflags.csv',encoding = "ISO-8859-1")[:8398]#last entry that contains country info
#temperature change not absolute values
#exclude years before 1996
rem_cols = [c for c in temp0.columns if c[:3] == "Y19"][:-4]
temp0.drop(columns = rem_cols, inplace = True)
#Remove the Y in front of years
columns_old = temp0.columns[7:]
columns_new = [c[1:] for c in temp0.columns[7:]]
temp0.rename(columns = dict(zip(columns_old, columns_new)),
          inplace=True, errors='raise')
#lets remove some columns
rem_cols = [c for c in temp0.columns if (c == "Unit" or c=='Element Code' or c=='Area Code')  ]
temp0.drop(columns = rem_cols, inplace = True)
#temp0 = temp0.T.fillna(temp0.mean(axis=1)).T#replace nan values with mean
print('unique val: ', temp0['Area'].nunique())
temp0.head(n = 3)

unique val:  247


,Area,Months Code,Months,Element,1996,1997,1998,1999,2000,2001,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,7001,January,Temperature change,-0.718,1.426,0.95,0.859,1.565,-0.603,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,Afghanistan,7001,January,Standard Deviation,1.950,1.950,1.95,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,Afghanistan,7002,February,Temperature change,0.870,0.043,-0.54,3.222,-0.901,0.707,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086


## Reshape

In [818]:
#let's take years as one column
temp1 = temp0.melt(id_vars=temp0.columns[:4], 
        var_name="Year", 
        value_name="Value")
temp1.head(n=1)

,Area,Months Code,Months,Element,Year,Value
0,Afghanistan,7001,January,Temperature change,1996,-0.718


In [819]:
#the elment column entries should be columns
temp2 = temp1.set_index(['Area','Months Code','Months', 'Year', 'Element'])['Value'].unstack()
#move the multiindex one down
temp2 = temp2.reset_index().rename_axis(None).rename_axis(None, axis=1)
#rename area to country
temp2 = temp2.rename(columns={temp2.columns[0]: 'Country'})
#remove std column
rem_cols = [c for c in temp2.columns if c == "Standard Deviation"]
temp2.drop(columns = rem_cols, inplace = True)
temp2.head(n =2)

,Country,Months Code,Months,Year,Temperature change
0,Afghanistan,7001,January,1996,-0.718
1,Afghanistan,7001,January,1997,1.426


## Select annual average 

In [820]:
annual_mask = temp2['Months']=='Meteorological year'
temp3 = temp2[annual_mask]
#now we can drop Months and Months Code
temp3.drop(columns = ['Months Code', 'Months'], inplace = True)
temp3.head(3)

D:\Programme\Anaconda\envs\tfCPU\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Year,Temperature change
384,Afghanistan,1996,-0.106
385,Afghanistan,1997,0.480
386,Afghanistan,1998,0.669


## Lets compare the country entries to the A3 column before merging

In [821]:
#merge to see which country entries do not match
mergedStuff = pd.merge(temp3, alpha3, on=['Country'], how='inner')
mergedStuff.head()
merged_countries = mergedStuff['Country'].unique()

In [822]:
countries_in_both_mask_alpha= alpha3['Country'].isin(merged_countries)
diff_countries_alpha = alpha3[~countries_in_both_mask_alpha]['Country'].unique()
diff_countries_alpha

array(['Åland Islands', 'Bermuda', 'Bonaire, Sint Eustatius and Saba[a]',
       'Bouvet Island', 'British Indian Ocean Territory',
       'Congo, Democratic Republic of the', 'Curaçao',
       'French Southern Territories', 'Guam', 'Guernsey',
       'Heard Island and McDonald Islands', 'Hong Kong', 'Jersey',
       "Korea (Democratic People's Republic of)", 'Korea, Republic of',
       'Macao', 'Moldova, Republic of', 'Northern Mariana Islands',
       'Palestine, State of', 'Pitcairn', 'Saint Barthélemy',
       'Saint Helena, Ascension and Tristan da Cunha[b]',
       'Saint Martin (French part)', 'Sint Maarten (Dutch part)',
       'Svalbard and Jan Mayen[c]', 'Taiwan, Province of China',
       'Tanzania, United Republic of',
       'United Kingdom of Great Britain and Northern Ireland',
       'United States Minor Outlying Islands[e]',
       'Virgin Islands (British)', 'Virgin Islands (U.S.)',
       'Wallis and Futuna'], dtype=object)

In [823]:
#find other countries:
countries_in_both_mask= temp3['Country'].isin(merged_countries)
diff_countries = temp3[~countries_in_both_mask]['Country'].unique()
print(diff_countries)
#the last 17 entries should be removed since these are not countries
diff_countries = diff_countries[[3,4,5,8,9,15,17,18,19,22,24,25,26,28]]
print(diff_countries)
#diff_countries = diff_countries[:-18]
#print(diff_countries)

['Belgium-Luxembourg' 'British Virgin Islands' 'Channel Islands'
 'China, Hong Kong SAR' 'China, Macao SAR' 'China, Taiwan Province of'
 'China, mainland' 'Czechoslovakia'
 "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Ethiopia PDR'
 'French Southern and Antarctic Territories' 'Midway Island'
 'Netherlands Antilles (former)' 'Pacific Islands Trust Territory'
 'Palestine' 'Pitcairn Islands' 'Republic of Korea' 'Republic of Moldova'
 'Saint Helena, Ascension and Tristan da Cunha' 'Serbia and Montenegro'
 'Sudan (former)' 'Svalbard and Jan Mayen Islands' 'USSR' 'United Kingdom'
 'United Republic of Tanzania' 'United States Virgin Islands'
 'Wake Island' 'Wallis and Futuna Islands' 'Yugoslav SFR']
['China, Hong Kong SAR' 'China, Macao SAR' 'China, Taiwan Province of'
 "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Palestine' 'Republic of Korea'
 'Republic of Moldova' 'Saint Helena, Ascension and Tristan da Cunha'
 'Svalbard and J

In [824]:
#Countries that have a different string in temp2 and A3 table
A3_code_temp = [
    'Hong Kong', 'Macao', "Taiwan, Province of China", "Korea (Democratic People's Republic of)",
    'Congo, Democratic Republic of the', 'Palestine, State of', 'Korea, Republic of',
    'Moldova, Republic of', 'Saint Helena, Ascension and Tristan da Cunha[b]',
    'Svalbard and Jan Mayen[c]', 'United Kingdom of Great Britain and Northern Ireland',
    'Tanzania, United Republic of', 'Virgin Islands (U.S.)','Wallis and Futuna'
    ]

In [825]:
#Replace non-matching countries with entries from the alpha3 df
di = dict(zip(diff_countries,A3_code_temp))
temp3['Country'] = temp3['Country'].map(di).fillna(temp3['Country'])
temp3.head(n=2)

D:\Programme\Anaconda\envs\tfCPU\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Year,Temperature change
384,Afghanistan,1996,-0.106
385,Afghanistan,1997,0.480


In [826]:
temp = temp3.merge(alpha3, on = 'Country')#add A3 and 
temp.head(n=2)

,Country,Year,Temperature change,A3
0,Afghanistan,1996,-0.106,AFG
1,Afghanistan,1997,0.480,AFG


In [827]:
#turn years into integers
temp['Year'] = temp['Year'].apply(pd.to_numeric, errors = 'coerce')
Input3 = temp.merge(Input2, how = 'outer', on = ['A3', 'Year'])
Input3.head(n=2)

,Country,Year,Temperature change,A3,HDI,Population,Refugees
0,Afghanistan,1996,-0.106,AFG,0.335,18853437.0,2674234.0
1,Afghanistan,1997,0.480,AFG,0.339,19357126.0,2676675.0


In [828]:
Input3.columns

Index(['Country', 'Year', 'Temperature change', 'A3', 'HDI', 'Population',
       'Refugees'],
      dtype='object')

# Governance Indices 

In [829]:
wgi0 = pd.read_csv('Data/WGI/WGIData.csv')
#last column is unnamed and empty
wgi0.drop(columns = wgi0.columns[-1:], inplace = True)

#years 1997,1999 anmd 2001 are missing, lets average over the following and preceding year
#the test set is not affected by it!
wgi0['1997'] = wgi0[['1996', '1998']].mean(axis = 1)
cols = wgi0.columns.tolist()
cols.insert(5, cols.pop(-1))# move it to the right position

wgi0 = wgi0[cols]
wgi0['2001'] = wgi0[['2000', '2002']].mean(axis = 1)
cols = wgi0.columns.tolist()
cols.insert(8, cols.pop(-1))# move it to the right position

wgi0 = wgi0[cols]
wgi0['1999'] = wgi0[['1998', '2000']].mean(axis = 1)
cols = wgi0.columns.tolist()
cols.insert(7, cols.pop(-1))# move it to the right position


wgi0 = wgi0[cols]
wgi0.drop(columns = wgi0.columns[[0,3]], inplace = True)
wgi0.head(n=3)

,Country Code,Indicator Name,1996,1997,1998,1999,2000,2001,2002,2003,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AFG,Control of Corruption: Estimate,-1.291705,-1.236276,-1.180848,-1.238114,-1.295380,-1.279373,-1.263366,-1.351042,...,-1.635723,-1.579179,-1.419888,-1.436761,-1.354784,-1.341994,-1.526352,-1.517361,-1.496834,-1.401076
1,AFG,Control of Corruption: Number of Sources,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,...,9.000000,9.000000,10.000000,11.000000,11.000000,11.000000,10.000000,10.000000,10.000000,10.000000
2,AFG,Control of Corruption: Percentile Rank,4.301075,7.047445,9.793815,7.434978,5.076142,5.063324,5.050505,5.050505,...,0.952381,0.947867,2.369668,1.895735,5.288462,6.250000,3.365385,3.846154,4.326923,6.730769


In [830]:
#there are nan values, replace with mean over years
#!this might be not the best approach, maybe just setting estimates to a constant and std very high is better
#if it can be included in training
#wgi1 = wgi0.T.fillna(wgi0.mean(axis=1)).T

## Reshape

In [831]:
#let's take years as one column
wgi2 = wgi0.melt(id_vars=wgi0.columns[:2], 
        var_name="Year", 
        value_name="Value")
wgi2.head(n = 2)

,Country Code,Indicator Name,Year,Value
0,AFG,Control of Corruption: Estimate,1996,-1.291705
1,AFG,Control of Corruption: Number of Sources,1996,2.000000


In [832]:
#the elment column entries should be columns
wgi3 = wgi2.set_index(['Country Code','Year','Indicator Name'])['Value'].unstack()
#move the multiindex one down
wgi3 = wgi3.reset_index().rename_axis(None).rename_axis(None, axis=1)
wgi3.head(n=2)

,Country Code,Year,Control of Corruption: Estimate,Control of Corruption: Number of Sources,Control of Corruption: Percentile Rank,"Control of Corruption: Percentile Rank, Lower Bound of 90% Confidence Interval","Control of Corruption: Percentile Rank, Upper Bound of 90% Confidence Interval",Control of Corruption: Standard Error,Government Effectiveness: Estimate,Government Effectiveness: Number of Sources,...,Rule of Law: Percentile Rank,"Rule of Law: Percentile Rank, Lower Bound of 90% Confidence Interval","Rule of Law: Percentile Rank, Upper Bound of 90% Confidence Interval",Rule of Law: Standard Error,Voice and Accountability: Estimate,Voice and Accountability: Number of Sources,Voice and Accountability: Percentile Rank,"Voice and Accountability: Percentile Rank, Lower Bound of 90% Confidence Interval","Voice and Accountability: Percentile Rank, Upper Bound of 90% Confidence Interval",Voice and Accountability: Standard Error
0,ABW,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [833]:
wgi4 = wgi3.rename(columns={wgi3.columns[0]: 'A3', })
#lets drop some columns
#remove confidence intervals and number of sources
wgi4.drop(columns = wgi4.columns[[3,5,6, 9,11, 12,15, 17, 18,21, 23, 24,27, 29, 30,33, 35, 36]], inplace = True)
print('nunique: ',wgi4['A3'].nunique())
wgi4.head(n=2)

nunique:  214


,A3,Year,Control of Corruption: Estimate,Control of Corruption: Percentile Rank,Control of Corruption: Standard Error,Government Effectiveness: Estimate,Government Effectiveness: Percentile Rank,Government Effectiveness: Standard Error,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Percentile Rank,Political Stability and Absence of Violence/Terrorism: Standard Error,Regulatory Quality: Estimate,Regulatory Quality: Percentile Rank,Regulatory Quality: Standard Error,Rule of Law: Estimate,Rule of Law: Percentile Rank,Rule of Law: Standard Error,Voice and Accountability: Estimate,Voice and Accountability: Percentile Rank,Voice and Accountability: Standard Error
0,ABW,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Before merging with the input we should only keep standard A3 values
lets compare which differ:


In [834]:
#merge to see which country entries do not match
mergedStuff = pd.merge(wgi, alpha3, on=['A3'], how='inner')
mergedStuff.head()
merged_countries = mergedStuff['A3'].unique()
#find other countries:
countries_in_both_mask= wgi['A3'].isin(merged_countries)
diff_countries = wgi[~countries_in_both_mask]['A3'].unique()
print(diff_countries)
#we just exclude these to values from wgi
wgi = wgi[~wgi['A3'].isin(diff_countries)]

['ANT' 'XKX']


## Drop columns and merge 

In [835]:
#We can drop percentile ranks, since the model can learn it just from the estimates.
wgi = wgi4.drop(columns = wgi4.columns[[3, 6,9,12,15,18]])
#turn years into integers
wgi['Year'] = wgi['Year'].apply(pd.to_numeric, errors = 'coerce')
#Merge with the input
Input4 = wgi.merge(Input3, how = 'outer', on = ['A3', 'Year'])
print('nunique: ', Input4['A3'].nunique())
Input4.head(n=2)

nunique:  239


,A3,Year,Control of Corruption: Estimate,Control of Corruption: Standard Error,Government Effectiveness: Estimate,Government Effectiveness: Standard Error,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error,Regulatory Quality: Estimate,Regulatory Quality: Standard Error,Rule of Law: Estimate,Rule of Law: Standard Error,Voice and Accountability: Estimate,Voice and Accountability: Standard Error,Country,Temperature change,HDI,Population,Refugees
0,ABW,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aruba,0.875,NaN,NaN,NaN
1,ABW,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aruba,0.551,NaN,NaN,NaN


In [836]:
#Lets drop Country from input, we can look that up using alpha3
Input = Input4.drop(columns=['Country'])

In [837]:
Input[Input['A3'].isnull()]#No nan values

,A3,Year,Control of Corruption: Estimate,Control of Corruption: Standard Error,Government Effectiveness: Estimate,Government Effectiveness: Standard Error,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error,Regulatory Quality: Estimate,Regulatory Quality: Standard Error,Rule of Law: Estimate,Rule of Law: Standard Error,Voice and Accountability: Estimate,Voice and Accountability: Standard Error,Temperature change,HDI,Population,Refugees


In [838]:
Input.columns

Index(['A3', 'Year', 'Control of Corruption: Estimate',
       'Control of Corruption: Standard Error',
       'Government Effectiveness: Estimate',
       'Government Effectiveness: Standard Error',
       'Political Stability and Absence of Violence/Terrorism: Estimate',
       'Political Stability and Absence of Violence/Terrorism: Standard Error',
       'Regulatory Quality: Estimate', 'Regulatory Quality: Standard Error',
       'Rule of Law: Estimate', 'Rule of Law: Standard Error',
       'Voice and Accountability: Estimate',
       'Voice and Accountability: Standard Error', 'Temperature change', 'HDI',
       'Population', 'Refugees'],
      dtype='object')

# Lets save the input

In [839]:
Input.to_csv('Data/Input.csv', index = False)